In [1]:
%load_ext autoreload
%autoreload 2

from itertools import islice, product
from typing import Any, Callable, Optional, Sequence, Type, Union

import scipy.io as sio
import torch
from icecream import ic

from mrboost import reconstruction as recon

/bmrc-homes/nmrgrp/nmr201/micromamba/envs/python311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_data, shape_dict, mdh, twixobj = recon.get_raw_data(
    "/bmrc-an-data/Chunwei/WIP/Test_DCE_SoS/RawData/Subject_1/meas_MID00255_FID13072_fl3d_vibe_GA_2_320.dat"
)

pymapVBVD version 0+untagged.22.gac0eb13.dirty
Software version: VD


read data:  71%|███████   | 10855/15360 [00:11<00:04, 978.60it/s] /bmrc-an-data/Chunxu/mrboost/src/mrboost/mapvbvd/twix_map_obj.py:870: RuntimeWarning: invalid value encountered in add
  raw = (raw[:, 0] + 1j * raw[:, 1]).reshape(readShape, order="F")


{'ch_num': 52, 'partition_num': 48, 'spoke_num': 320, 'spoke_len': 384}


In [3]:
ga_args = recon.DynGoldenAngleVibeArgs(
    shape_dict, mdh, twixobj, contra_num=1, spokes_per_contra=160
)

In [4]:
preprocessed_data = recon.preprocess_raw_data(raw_data, ga_args)

100%|██████████| 13/13 [00:03<00:00,  4.06it/s]


In [5]:
for k, v in preprocessed_data.items():
    print(k, v.shape)

kspace_data_centralized torch.Size([1, 52, 60, 160, 384])
kspace_data_z torch.Size([1, 52, 60, 160, 384])
kspace_traj torch.Size([1, 2, 160, 384])


In [8]:
image = recon.mcnufft_reconstruct(preprocessed_data, ga_args, False)

ic| 'reconstructing contrast:', t: 0
/bmrc-homes/nmrgrp/nmr201/micromamba/envs/python311/lib/python3.11/site-packages/jax/_src/dlpack.py:71: UserWarning: take_ownership in to_dlpack is deprecated and it is a no-op.
  warnings.warn(


In [20]:
from scipy.io import savemat


savemat(
    "/bmrc-an-data/Chunxu/MR_Recon/Subject_1/meas_MID00255_FID13072_fl3d_vibe_GA_2_320.mat",
    {"image": image.abs()},
)